In [1]:
from PIL import Image
import os

# Reemplaza esto con la ruta de la carpeta que contiene tus imágenes
base_folder = './test'

# Recorremos todas las subcarpetas y archivos en la carpeta base
for subdir, dirs, files in os.walk(base_folder):
    for file in files:
        # Comprobamos si el archivo es una imagen
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(subdir, file)
            with Image.open(file_path) as img:
                # Calculamos las dimensiones del recorte
                width, height = img.size
                left = width * 0.05
                right = width - left
                top = 0
                bottom = height * 0.9

                # Recortamos la imagen
                img_cropped = img.crop((left, top, right, bottom))

                # Guardamos la imagen recortada, posiblemente con un nuevo nombre o en una nueva ubicación
                img_cropped.save(file_path.replace('.png', '_cropped.png').replace('.jpg', '_cropped.jpg').replace('.jpeg', '_cropped.jpeg'))
                # O sobrescribimos la imagen original (¡ten cuidado con esto!)
                # img_cropped.save(file_path)


In [2]:
import os
from PIL import Image
import numpy as np

# Definimos las rutas a los folders de imágenes y máscaras
images_folder = './test/images'
masks_folder = './test/masks'

# Función para procesar las imágenes y máscaras
def crop_images_based_on_masks(images_folder, masks_folder):
    cropped_images_paths = []
    # Asumimos que los nombres de archivos de las máscaras e imágenes son los mismos
    for image_file in os.listdir(images_folder):
        if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue  # Si no es una imagen, saltamos al siguiente archivo

        image_path = os.path.join(images_folder, image_file)
        mask_path = os.path.join(masks_folder, image_file)

        # Cargamos la imagen y su máscara correspondiente
        image = Image.open(image_path)
        mask = Image.open(mask_path)

        # Convertimos la máscara a un array binario
        mask_array = np.array(mask)
        binary_mask = mask_array > 128  # Asumimos que la máscara es una imagen en escala de grises

        # Obtenemos las coordenadas del bounding box de la máscara binaria
        rows = np.any(binary_mask, axis=1)
        cols = np.any(binary_mask, axis=0)
        ymin, ymax = np.where(rows)[0][[0, -1]]
        xmin, xmax = np.where(cols)[0][[0, -1]]

        # Recortamos la imagen de entrada con el bounding box de la máscara
        cropped_image = image.crop((xmin, ymin, xmax, ymax))

        # Guardamos la imagen recortada
        cropped_image_path = image_path.replace('.png', '_cropped.png').replace('.jpg', '_cropped.jpg').replace('.jpeg', '_cropped.jpeg')
        cropped_image.save(cropped_image_path)
        cropped_images_paths.append(cropped_image_path)

    return cropped_images_paths

# Llamamos a la función y pasamos las rutas de los folders
cropped_images_paths = crop_images_based_on_masks(images_folder, masks_folder)

# Imprimir las rutas de las imágenes recortadas para verificación
print(cropped_images_paths)


['./test/images\\COVID-1_cropped.png', './test/images\\COVID-2_cropped.png', './test/images\\COVID-3_cropped.png', './test/images\\COVID-4_cropped.png', './test/images\\COVID-5_cropped.png']


In [6]:
import os
from PIL import Image
import numpy as np

# Definimos las rutas a los folders de imágenes y máscaras
images_folder = './test/images'
masks_folder = './test/masks'
crop_folder = './test/crop'

# Función para procesar las imágenes y máscaras
def crop_images_based_on_masks(images_folder, masks_folder):
    cropped_images_paths = []
    for image_file in os.listdir(images_folder):
        if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue  # Si no es una imagen, saltamos al siguiente archivo

        image_path = os.path.join(images_folder, image_file)
        mask_path = os.path.join(masks_folder, image_file)
        crop_path = os.path.join(crop_folder, image_file)

        # Cargamos la imagen y su máscara correspondiente
        image = Image.open(image_path)
        mask = Image.open(mask_path)

        # Convertimos la máscara a un array binario
        mask_array = np.array(mask)
        binary_mask = mask_array > 128  # Asumimos que la máscara es una imagen en escala de grises

        # Obtenemos las coordenadas del bounding box de la máscara binaria
        rows = np.any(binary_mask, axis=1)
        cols = np.any(binary_mask, axis=0)
        ymin, ymax = np.where(rows)[0][[0, -1]]
        xmin, xmax = np.where(cols)[0][[0, -1]]

        # Ajustamos el recorte para dejar un 5% adicional en la parte inferior y el lado derecho
        height, width = image.size
        ymax_adj = min(ymax + int(height * 0.05), height)  # Aseguramos que no exceda el tamaño de la imagen
        xmax_adj = min(xmax + int(width * 0.05), width)

        # Recortamos la imagen de entrada con el bounding box ajustado
        cropped_image = image.crop((xmin, ymin, xmax_adj, ymax_adj))

        # Guardamos la imagen recortada
        cropped_image_path = crop_path.replace('.png', '_cropped.png').replace('.jpg', '_cropped.jpg').replace('.jpeg', '_cropped.jpeg')
        cropped_image.save(cropped_image_path)
        cropped_images_paths.append(cropped_image_path)

    return cropped_images_paths

# Llamamos a la función y pasamos las rutas de los folders
cropped_images_paths = crop_images_based_on_masks(images_folder, masks_folder)

# Imprimir las rutas de las imágenes recortadas para verificación
print(cropped_images_paths)


FileNotFoundError: [Errno 2] No such file or directory: './test/masks\\COVID-1_cropped.png'

In [7]:
import os
from PIL import Image
import numpy as np

# Definimos las rutas a los folders de imágenes y máscaras
images_folder = './test/images'
masks_folder = './test/masks'
crop_folder = './test/crop'

# Crea el directorio de recortes si no existe
if not os.path.exists(crop_folder):
    os.makedirs(crop_folder)

# Función para procesar las imágenes y máscaras
def crop_images_based_on_masks(images_folder, masks_folder, crop_folder):
    cropped_images_paths = []
    for image_file in os.listdir(images_folder):
        if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue  # Si no es una imagen, saltamos al siguiente archivo

        image_path = os.path.join(images_folder, image_file)
        mask_path = os.path.join(masks_folder, image_file)
        crop_path = os.path.join(crop_folder, image_file)

        try:
            # Cargamos la imagen y su máscara correspondiente
            image = Image.open(image_path)
            mask = Image.open(mask_path)
        except FileNotFoundError:
            print(f"No se encontró el archivo para {image_file}")
            continue

        # Convertimos la máscara a un array binario
        mask_array = np.array(mask)
        binary_mask = mask_array > 128  # Asumimos que la máscara es una imagen en escala de grises

        # Obtenemos las coordenadas del bounding box de la máscara binaria
        rows = np.any(binary_mask, axis=1)
        cols = np.any(binary_mask, axis=0)
        ymin, ymax = np.where(rows)[0][[0, -1]]
        xmin, xmax = np.where(cols)[0][[0, -1]]

        # Ajustamos el recorte para dejar un 5% adicional en la parte inferior y el lado derecho
        height, width = image.size
        ymax_adj = min(ymax + int(height * 0.05), height)  # Aseguramos que no exceda el tamaño de la imagen
        xmax_adj = min(xmax + int(width * 0.05), width)

        # Recortamos la imagen de entrada con el bounding box ajustado
        cropped_image = image.crop((xmin, ymin, xmax_adj, ymax_adj))

        # Guardamos la imagen recortada
        cropped_image_path = crop_path.replace('.png', '_cropped.png').replace('.jpg', '_cropped.jpg').replace('.jpeg', '_cropped.jpeg')
        cropped_image.save(cropped_image_path)
        cropped_images_paths.append(cropped_image_path)

    return cropped_images_paths

# Llamamos a la función y pasamos las rutas de los folders
cropped_images_paths = crop_images_based_on_masks(images_folder, masks_folder, crop_folder)

# Imprimir las rutas de las imágenes recortadas para verificación
print(cropped_images_paths)


No se encontró el archivo para COVID-1_cropped.png
No se encontró el archivo para COVID-1_cropped_v2.png
['./test/crop\\COVID-1_cropped.png', './test/crop\\COVID-2_cropped.png', './test/crop\\COVID-3_cropped.png', './test/crop\\COVID-4_cropped.png', './test/crop\\COVID-5_cropped.png']
